In [31]:
import multiprocessing
import os
import time

%matplotlib inline

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [15]:
DATA_PATH = "/home/wangc21/datasets/pool/images"

In [21]:
class PoolDataset(torch.utils.data.Dataset):
    def __init__(self, label_path, transform = None):
        self.labels = pd.read_csv(label_path, sep = ",", header = None).values
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_idx, label = self.labels[idx][0], self.labels[idx][1]
        image = cv2.imread(os.path.join(DATA_PATH, img_idx))
        return (image, label)

In [22]:
# data augmentation

train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees = 360),
    transforms.ColorJitter(brightness = 0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

data_train = PoolDataset(os.path.join(DATA_PATH, "train_labels.csv"), transform = train_transforms)
print(len(data_train))
data_test = PoolDataset(os.path.join(DATA_PATH, "test_labels.csv"), transform = test_transforms)
print(len(data_test))

12824
3176


In [37]:
class Darknet(nn.Module):
    def __init__(self):
        super(Darknet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1, 1)
        self.conv4 = nn.Conv2d(64, 128, 3, 1, 1)
        self.conv5 = nn.Conv2d(128, 256, 3, 1, 1)
        #self.conv6 = nn.Conv2d(256, 512, 3, 1, 1)
        #self.conv7 = nn.Conv2d(512, 1024, 3, 1, 1)
        self.fc = nn.Linear(256, 16)
        
    def forward(self, x):
        # HWC to CHW
        x = x.permute(0, 3, 1, 2)
        
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, 2, 2)
        #x = F.relu(self.conv6(x))
        #x = F.max_pool2d(x, 2, 2)
        #x = F.relu(self.conv7(x))
        #x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 256)
        x = self.fc(x)
        return x
    
    def loss(self, prediction, label, reduction = "mean"):
        loss = F.cross_entropy(prediction, label, reduction = reduction)
        return loss

In [38]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []
    
    # stochastic gradient descent
    for batch_idx, (data, label) in enumerate(train_loader):
        data = data.to(device = device, dtype = torch.float)
        label = label.to(device = device, dtype = torch.long)

        # reset gradients
        optimizer.zero_grad()
        
        # make prediction
        output = model(data)
        
        # compute error gradients
        loss = model.loss(output, label)
        losses.append(loss.item())
        loss.backward()
        
        # update weights
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('{} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                time.ctime(time.time()),
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    return np.mean(losses)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(test_loader):
            data = data.to(device = device, dtype = torch.float)
            label = label.to(device = device, dtype = torch.long)
            output = model(data)
            test_loss += model.loss(output, label, reduction = "sum").item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return test_loss

In [39]:
# hyperparameters
EPOCHS = 200
TRAIN_BATCH_SIZE = 200
TEST_BATCH_SIZE = 50
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0005

# hardware device
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': multiprocessing.cpu_count(),
          'pin_memory': True} if use_cuda else {}

# data loaders
train_loader = torch.utils.data.DataLoader(data_train, batch_size = TRAIN_BATCH_SIZE,
                                           shuffle = True, drop_last = True, **kwargs)
test_loader = torch.utils.data.DataLoader(data_test, batch_size = TEST_BATCH_SIZE,
                                          shuffle = False, drop_last = True, **kwargs)  

# training objects
model = Darknet().to(device)
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY)

# training loop
train_loss = []
test_loss = []
best_loss = np.inf
for epoch in range(1, EPOCHS + 1):
    train_l = train(model, device, train_loader, optimizer, epoch)
    test_l = test(model, device, test_loader)
    train_loss.append(train_l)
    test_loss.append(test_l)
    
    # save best model
    if test_l < best_loss:
        best_loss = test_l
        torch.save(model.state_dict(), "epoch_%d.pt" % epoch)
        
# plot loss functions
plt.plot(np.arange(EPOCHS) + 1, train_loss, label = "train")
plt.plot(np.arange(EPOCHS) + 1, test_loss, label = "test")
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

Sun Jun  7 04:45:48 2020 Train Epoch: 1 [0/12824 (0%)]	Loss: 4.309289

Test set: Average loss: 0.1633, Accuracy: 0/3176 (0%)

Sun Jun  7 04:45:49 2020 Train Epoch: 2 [0/12824 (0%)]	Loss: 0.195869

Test set: Average loss: 0.0214, Accuracy: 0/3176 (0%)

Sun Jun  7 04:45:51 2020 Train Epoch: 3 [0/12824 (0%)]	Loss: 0.017004

Test set: Average loss: 0.0288, Accuracy: 0/3176 (0%)

Sun Jun  7 04:45:52 2020 Train Epoch: 4 [0/12824 (0%)]	Loss: 0.022212

Test set: Average loss: 0.0101, Accuracy: 0/3176 (0%)

Sun Jun  7 04:45:54 2020 Train Epoch: 5 [0/12824 (0%)]	Loss: 0.003910

Test set: Average loss: 0.0051, Accuracy: 0/3176 (0%)

Sun Jun  7 04:45:55 2020 Train Epoch: 6 [0/12824 (0%)]	Loss: 0.007231

Test set: Average loss: 0.0019, Accuracy: 0/3176 (0%)

Sun Jun  7 04:45:57 2020 Train Epoch: 7 [0/12824 (0%)]	Loss: 0.000192

Test set: Average loss: 0.0063, Accuracy: 0/3176 (0%)

Sun Jun  7 04:45:58 2020 Train Epoch: 8 [0/12824 (0%)]	Loss: 0.000317

Test set: Average loss: 0.0012, Accuracy: 0/317

Process Process-1037:


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/wangc21/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/wangc21/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wangc21/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 199, in _worker_loop
    data_queue.close()
  File "/home/wangc21/anaconda3/lib/python3.7/multiprocessing/queues.py", line 139, in close
    close()
  File "/home/wangc21/anaconda3/lib/python3.7/multiprocessing/util.py", line 183, in __call__
    if self._pid != getpid():
KeyboardInterrupt
